In [1]:
using Makie

In [2]:
function LJ(r:: Float64)
    return (1/r^12 - 2/r^6) 
end

# 微分形
function LJforce(r::Float64)
    return 12*(1/r^14-1/r^8)
end

function force(x,y,z,N)
    ax = Vector{Float64}(undef,N)
    ay = Vector{Float64}(undef,N)
    az = Vector{Float64}(undef,N)

    ax .= 0
    ay .= 0
    az .= 0
    
    for i in 1:N-1
        for j in i+1:N
            ll = sqrt((x[j]-x[i])^2 + (y[j]-y[i])^2 + (z[j]-z[i])^2)
            FLJ = LJforce(ll)
            
            ax[i] -= FLJ * (x[j]-x[i])
            ax[j] += FLJ * (x[j]-x[i])
            ay[i] -= FLJ * (y[j]-y[i])
            ay[j] += FLJ * (y[j]-y[i])
            az[i] -= FLJ * (z[j]-z[i])
            az[j] += FLJ * (z[j]-z[i])
        end
    end
    
    return ax, ay, az
end

function ルンゲクッタ(x,y,z,u,v,w,N;
        T=20., dt=1e-3)
    
    numStep = length(0:dt:T)
    
    xlist = Array{Float64,2}(undef,N,numStep)
    ylist = Array{Float64,2}(undef,N,numStep)
    zlist = Array{Float64,2}(undef,N,numStep)
    ulist = Array{Float64,2}(undef,N,numStep)
    vlist = Array{Float64,2}(undef,N,numStep)
    wlist = Array{Float64,2}(undef,N,numStep) 
    for (i,t) in enumerate(0:dt:T)
        # リストに書き込む
        xlist[:,i] = x
        ylist[:,i] = y
        zlist[:,i] = z
        ulist[:,i] = u
        vlist[:,i] = v
        wlist[:,i] = w
        # 微小変化の計算
        # k1
        dx1 = u
        dy1 = v
        dz1 = w
        du1, dv1, dw1 = force(x,y,z,N)
        
        # k2
        x1 = x .+ .5dt.*dx1 
        y1 = y .+ .5dt.*dy1
        z1 = z .+ .5dt.*dz1 
    u1 = u .+ .5dt.*du1
        v1 = v .+ .5dt.*dv1
        w1 = w .+ .5dt.*dw1
        dx2 = u1
        dy2 = v1
        dz2 = w1
        du2, dv2, dw2 = force(x1,y1,z1,N) 
        
        # k3
        x2 = x .+ .5dt.*dx2 
        y2 = y .+ .5dt.*dy2
        z2 = z .+ .5dt.*dz2
        u2 = u .+ .5dt.*du2
        v2 = v .+ .5dt.*dv2
        w2 = w .+ .5dt.*dw2
        dx3 = u2
        dy3 = v2
    dz3 = w2
        du3, dv3, dw3 = force(x2,y2,z2,N)
        
        # k4
        x3 = x + dt*dx3 
        y3 = y + dt*dy3
    z3 = z + dt*dz3
        u3 = u + dt*du3
        v3 = v + dt*dv3
        w3 = w + dt*dw3
        dx4 = u3
        dy4 = v3
    dz4 = w3
        du4, dv4, dw4 = force(x3,y3,z3,N)
        
        # がっちゃんこ
        x += dt*(dx1 + 2dx2 + 2dx3 + dx4)/6. 
        y += dt*(dy1 + 2dy2 + 2dy3 + dy4)/6.
        z += dt*(dz1 + 2dz2 + 2dz3 + dz4)/6.
    
        u += dt*(du1 + 2du2 + 2du3 + du4)/6.
        v += dt*(dv1 + 2dv2 + 2dv3 + dv4)/6.
        w += dt*(dw1 + 2dw2 + 2dw3 + dw4)/6.
        
  end
  return 0:dt:T, xlist, ylist,zlist, ulist, vlist, wlist
end

ルンゲクッタ (generic function with 1 method)

In [4]:
K = 10
L = 10
M = 18
N = K*L*M

x = Vector{Float64}(undef, N)
y = copy(x)
z = copy(x)

x[1]=0
y[1]=0
z[1]=0

I = 0
for i in 1:K
  for j in 1:L
    for k in 1:M
      I+=1
      x[I] = i*1.
      y[I] = j*1.
      z[I] = k*1.
  
    end
  end
end

N

1800

In [5]:

# 速度成分入力にも出力にもいらないのでは?
u = zeros(Float64, N)
v = copy(u)
w = copy(u)

@time tl,xl,yl,zl,ul,vl ,wl= ルンゲクッタ(x, y,z, u, v,w, N;T=50,dt=1e-2);

6107.250434 seconds (2.69 M allocations: 5.382 GiB, 0.02% gc time)


In [10]:
skip2 = 10
scene = Scene(resolution = (1000,1000))
mystep = Node(1)

scene = Makie.meshscatter!(
  scene, 
  lift(i -> xl[:,i],mystep),
  lift(i -> yl[:,i],mystep),
  lift(i -> zl[:,i],mystep),
  markersize = .5,
  color = :pink,
  limits =FRect3D(Vec3f0(0.,0.,0.), Vec3f0(M+1,M+1,M+1))
 #limits = FRect(-2. , -2., 1.5M+4., 1.5M+4.)
  )
@time record(scene, "muzan.mp4",1:skip2:length(tl)) do i
  #println(i)
  mystep[]=i
  end;

 32.192431 seconds (993.96 k allocations: 1.493 GiB, 1.54% gc time)
